In [9]:
import pandas as pd
import numpy as np
import io
import requests

In [10]:
%%bash 
url1="https://data.bitcoinity.org/export_data.csv?bp=1&bu=c&currency=USD&data_type=bidask_sum&exchange=coinbase&r=hour&t=m&timespan=30d"
url5="https://data.bitcoinity.org/export_data.csv?bp=5&bu=c&currency=USD&data_type=bidask_sum&exchange=coinbase&r=hour&t=m&timespan=30d"
url10="https://data.bitcoinity.org/export_data.csv?bp=10&bu=c&currency=USD&data_type=bidask_sum&exchange=coinbase&r=hour&t=m&timespan=30d"
curl $url1  --output bitcoinity_data1.csv
curl $url5  --output bitcoinity_data5.csv
curl $url10  --output bitcoinity_data10.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 52813    0 52813    0     0  32803      0 --:--:--  0:00:01 --:--:-- 32803
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 52791    0 52791    0     0  43306      0 --:--:--  0:00:01 --:--:-- 43271
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 52798    0 52798    0     0  37365      0 --:--:--  0:00:01 --:--:-- 37365


In [11]:
depths= [1,5,10]

def load_df(depth):
    df = pd.read_csv('bitcoinity_data{0}.csv'.format(depth))
    df[['asks', 'bids','price']] = df[['asks', 'bids','price']].apply(pd.to_numeric)
    df.index = pd.to_datetime(df['Time'])
    del df['Time']
    return df

def process_df(df, depth):
    df['bookratio{0}'.format(depth)] = np.log(df['bids']/df['asks'])
    return df

def combine_df(dfs):
    df=dfs[0]
    for i in range(1, len(depths)):
        depth=depths[i]
        df = df.join(dfs[i]['bookratio{0}'.format(depth)], on='Time')
    return df
        
dfs = [process_df(load_df(depth), depth) for depth in depths]
main_df=combine_df(dfs)

In [12]:
def order(row):
    x,y,z=row['bookratio1'], row['bookratio5'], row['bookratio10']
    if (x < y) and (y < z): #xyz
        return 0
    elif (y < x) and (x < z): #yxz
        return 1
    elif (x < z) and (z < y): #xzy
        return 2
    elif (y < z) and (z < x): #yzx
        return 3    
    elif (z < x) and (x < y): #zxy
        return 4
    elif (z < y) and (y < x): #zyx
        return 5
    
main_df['order'] = main_df.apply (lambda row: order(row), axis=1)

In [13]:
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [14]:
def color(row):
    order = row['order']
    num2color = {0:"green",1:"blue", 2:"blue", 3:"blue", 4:"blue", 5:"red"}
    return num2color[order]
colors = main_df.apply (lambda row: color(row), axis=1)

In [15]:
fig = go.Figure(data=go.Scatter(x=main_df.index, y=main_df['price'], mode='lines+markers', 
        marker=dict(
        color=colors,
        line_width=1)
        ))



In [16]:
fig.show()